In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pickle
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import csv
import warnings
warnings.simplefilter('ignore')

In [3]:
#sub_list = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17]
filename = '/content/drive/MyDrive/Colab Notebooks/preprocessed_data/2class/2_df_sub13.csv'
df = pd.read_csv(filename)
df = df.dropna(axis=0)
df

,c_ecg,c_emg,c_eda,c_temp,c_resp,w_label
0,-0.206268,-0.003983,2.383423,32.814331,0.721741,0.0
1,-0.237213,0.040970,2.416992,32.830811,0.750732,0.0
2,-0.269028,0.076492,2.375793,32.794861,0.776672,0.0
3,-0.287247,0.037903,2.373123,32.832306,0.762939,0.0
4,-0.296310,0.020004,2.376175,32.806824,0.787354,0.0
...,...,...,...,...,...,...
1558196,-0.014374,-0.002609,4.037094,34.319916,-7.708740,1.0
1558197,-0.017624,-0.002151,4.054260,34.322998,-7.704163,1.0
1558198,-0.015793,-0.000183,4.042435,34.327576,-7.708740,1.0
1558199,-0.009933,0.007278,4.037476,34.339813,-7.701111,1.0


In [4]:
X = df.drop('w_label', axis = 1)
y = df.w_label

In [5]:
X

,c_ecg,c_emg,c_eda,c_temp,c_resp
0,-0.206268,-0.003983,2.383423,32.814331,0.721741
1,-0.237213,0.040970,2.416992,32.830811,0.750732
2,-0.269028,0.076492,2.375793,32.794861,0.776672
3,-0.287247,0.037903,2.373123,32.832306,0.762939
4,-0.296310,0.020004,2.376175,32.806824,0.787354
...,...,...,...,...,...
1558196,-0.014374,-0.002609,4.037094,34.319916,-7.708740
1558197,-0.017624,-0.002151,4.054260,34.322998,-7.704163
1558198,-0.015793,-0.000183,4.042435,34.327576,-7.708740
1558199,-0.009933,0.007278,4.037476,34.339813,-7.701111


In [6]:
from sklearn.model_selection import train_test_split

In [7]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

In [8]:
check_for_nan = df.isnull().values.any()
print (check_for_nan)

False


In [9]:
import sklearn as sklearn
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import classification_report

In [10]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [11]:
pd.Series(y).value_counts()

0.0    1093401
1.0     464800
Name: w_label, dtype: int64

In [12]:
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_validate

# **Random Forest**

In [13]:
from sklearn.ensemble import RandomForestClassifier

In [14]:
forest = RandomForestClassifier(n_estimators = 50, min_samples_leaf = 5)

In [15]:
sfold = StratifiedKFold(n_splits = 10, random_state = 0, shuffle = True)
cross_validate(forest, x_train, y_train, 
               scoring=['accuracy', 'f1'], 
               return_train_score=True,
               cv = sfold)

{'fit_time': array([29.38732743, 30.87150335, 37.39636469, 30.0027318 , 31.63703752,
        30.21272397, 32.04728889, 28.83707476, 29.73364449, 30.77468657]),
 'score_time': array([0.35603809, 0.35666776, 0.37847471, 0.33681083, 0.35728216,
        0.3436408 , 0.38734221, 0.34917998, 0.34861398, 0.35796261]),
 'test_accuracy': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 'test_f1': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 'train_accuracy': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 'train_f1': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])}

# **Random Forest(SMOTE-ENN)**

In [16]:
from imblearn.combine import SMOTEENN

SMOTEENN = SMOTEENN(random_state=0)
x_train_over,y_train_over = SMOTEENN.fit_resample(x_train,y_train)

print('SMOTEENN 적용 전 학습용 피처/레이블 데이터 세트: ', x_train.shape, y_train.shape)
print('SMOTEENN 적용 후 학습용 피처/레이블 데이터 세트: ', x_train_over.shape, y_train_over.shape)
print('SMOTEENN 적용 후 레이블 값 분포: \n', pd.Series(y_train_over).value_counts())

SMOTEENN 적용 전 학습용 피처/레이블 데이터 세트:  (934920, 5) (934920,)
SMOTEENN 적용 후 학습용 피처/레이블 데이터 세트:  (1311636, 5) (1311636,)
SMOTEENN 적용 후 레이블 값 분포: 
 0.0    655818
1.0    655818
Name: w_label, dtype: int64


In [17]:
sfold = StratifiedKFold(n_splits = 10, random_state = 0, shuffle = True)
cross_validate(forest, x_train_over, y_train_over, 
               scoring=['accuracy', 'f1'], 
               return_train_score=True,
               cv = sfold)

{'fit_time': array([55.42314816, 56.00046563, 55.02902484, 56.62320566, 49.7956686 ,
        52.79038572, 46.19180751, 47.53059435, 47.9919889 , 43.0258162 ]),
 'score_time': array([0.44341946, 0.43562913, 0.46262169, 0.4361093 , 0.43114591,
        0.43790817, 0.45549488, 0.4456706 , 0.42041469, 0.44698977]),
 'test_accuracy': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 'test_f1': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 'train_accuracy': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 'train_f1': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])}